In [1]:
import torch
import torch.onnx
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 15.1 MB/s eta 0:00:00


In [2]:
# Загрузка предобученной модели LSTM
model = torch.nn.LSTM(10, 20, 2)

In [3]:
dummy_input = torch.randn(5, 3, 10)

In [4]:
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)

In [5]:
output, (hn, cn) = model(dummy_input, (h0, c0))

In [6]:
print("Результат запуска модели LSTM в PyTorch:", output)

Результат запуска модели LSTM в PyTorch: tensor([[[-1.1156e-01, -2.6123e-02, -6.8257e-02,  1.8778e-02, -2.3839e-02,
           1.2382e-01,  1.0813e-01, -1.3153e-01,  9.3796e-02,  3.7537e-01,
           2.6965e-01,  2.7565e-01,  3.5868e-01, -1.8160e-01, -4.3601e-02,
          -3.5078e-02, -2.6199e-02,  2.9819e-01,  1.4006e-01,  1.1159e-01],
         [-9.4034e-02,  3.3957e-02, -1.2563e-01,  7.7658e-02,  4.7857e-01,
          -5.0850e-01,  1.1084e-01, -1.3837e-01,  1.9291e-01,  3.8917e-02,
          -1.3782e-01,  2.2462e-02, -1.4602e-01,  3.8601e-01,  1.5102e-01,
           1.1777e-01, -2.4823e-01, -2.3651e-01, -4.5567e-02,  1.3717e-01],
         [ 1.9786e-01,  2.3278e-01,  1.3771e-01,  2.7722e-02,  3.4628e-01,
           2.2992e-01, -1.2608e-01, -3.3768e-01, -1.6091e-01, -1.9563e-01,
          -7.7813e-02,  4.4875e-01, -2.1314e-01, -3.6880e-01,  8.7306e-02,
           2.5998e-01,  4.1405e-02,  6.0621e-02, -2.2557e-02, -5.9181e-02]],

        [[-1.3917e-01,  6.5666e-02, -7.0182e-02,  8.01

In [7]:
# Сохранение модели в формате ONNX
torch.onnx.export(model, dummy_input, "lstm.onnx", verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [8]:
# Загрузка модели из файла ONNX
import onnx
import onnxruntime
onnx_model = onnx.load("lstm.onnx")
ort_session = onnxruntime.InferenceSession("lstm.onnx")

In [9]:
# Запуск модели из файла ONNX
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print("Результат запуска модели LSTM в ONNX:", ort_outs)

Результат запуска модели LSTM в ONNX: [array([[[-1.98980197e-02,  4.59743328e-02, -8.80215969e-03,
          6.76034838e-02, -4.77988124e-02, -1.04125012e-02,
         -6.10228814e-02,  4.33225892e-02, -1.47026405e-02,
          9.80176851e-02, -4.18041535e-02,  1.03707276e-02,
          4.48443973e-03,  3.18941511e-02, -1.65480133e-02,
         -1.81256346e-02,  2.30047945e-02, -9.09746101e-04,
         -4.94733751e-02,  2.51613725e-02],
        [-1.56341195e-02,  4.58927825e-02,  1.11150264e-04,
          6.87299073e-02, -2.01450288e-02, -1.03862565e-02,
         -5.99540956e-02,  1.16319517e-02, -1.06728952e-02,
          7.76977763e-02, -6.35566041e-02,  7.06979539e-03,
          5.29146241e-03,  3.88029590e-02, -3.37380134e-02,
         -3.99154536e-02,  9.14738420e-03,  1.90163718e-03,
         -4.77313697e-02,  1.97084155e-02],
        [-4.22157012e-02,  2.62648668e-02,  2.43022596e-03,
          7.13096857e-02, -4.93160710e-02, -2.07406171e-02,
         -5.52833527e-02,  2.5982

In [10]:
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("lstm.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.0 MB/s eta 0:00:00


NotImplementedError: ignored